# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural</font>

## GloVe: Global Vectors for Word2Vec

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


https://nlp.stanford.edu/pubs/glove.pdf

Os métodos para a aprendizagem de vetores de palavras se enquadram em uma das duas categorias: métodos baseados em fatoração de matriz global ou métodos baseados em janela de contexto local. 

A análise semântica latente (LSA, Latent Semantic Analysis) é um exemplo de um método baseado em fatoração de matriz global, e skip-gram e CBOW são métodos baseados em janela de contexto local. O LSA é usado como uma técnica de análise de documentos que mapeia palavras nos documentos para algo conhecido como conceito, um padrão comum de palavras que aparece em um documento. 

Os métodos baseados na fatoração da matriz global exploram eficientemente as estatísticas globais de um corpus (por exemplo, a co-ocorrência de palavras em um escopo global), mas mostram um desempenho ruim nas tarefas de analogia de palavras. Por outro lado, os métodos baseados em janelas de contexto mostraram ter um bom desempenho em tarefas de analogia de palavras, mas não utilizam estatísticas globais do corpus, deixando espaço para melhorias. 

O GloVe tenta obter o melhor dos dois mundos - uma abordagem que aproveita eficientemente as estatísticas de corpus globais, enquanto otimiza o modelo de aprendizado de uma maneira baseada em janelas de contexto, semelhante a skip-gram ou CBOW.

Word2vec e GloVe aprendem codificações geométricas (vetores) de palavras a partir de suas informações de co-ocorrência (com que frequência elas aparecem juntas em corpora de texto grande). Eles diferem em que word2vec é um modelo "preditivo", enquanto GloVe é um modelo "baseado em contagem".

Os modelos preditivos aprendem seus vetores a fim de melhorar sua capacidade preditiva (palavra-alvo | palavras de contexto; vetores), ou seja, a habilidade de predizer as palavras-alvo das palavras de contexto dadas às representações vetoriais. No Word2vec, isso é lançado como uma rede neural de feed-forward e otimizado como tal usando SGD, etc.

Modelos baseados em contagem aprendem seus vetores essencialmente fazendo a redução de dimensionalidade na matriz de contagem de co-ocorrência. Eles primeiro constroem uma grande matriz de informações de co-ocorrência (palavras x contexto), ou seja, para cada "palavra" (as linhas), você conta com que frequência vemos essa palavra em algum "contexto" (as colunas) em um corpus grande. O número de "contextos" é obviamente grande, uma vez que é essencialmente combinatório em tamanho. Então, eles fatorizam essa matriz para produzir uma matriz de dimensões inferiores (palavra x features), onde cada linha agora produz uma representação vetorial para cada palavra. Em geral, isso é feito minimizando uma "perda de reconstrução" que tenta encontrar as representações de menor dimensão que podem explicar a maior parte da variação nos dados de alta dimensão. No caso específico do GloVe, a matriz de contagens é pré-processada normalizando as contagens e e aplicando smoothing. Isso acaba sendo positivo em termos da qualidade das representações aprendidas.

No entanto, quando controlamos todos os hiperparâmetros de treinamento, as embeddings geradas usando os dois métodos tendem a ter um desempenho muito semelhante nas tarefas de PLN. Os benefícios adicionais do GloVe em relação ao Word2vec é que é mais fácil paralelizar a implementação, o que significa que é mais fácil treinar em mais dados, o que, com esses modelos, é sempre bom.

In [2]:
!nvidia-smi

Sat May 16 21:27:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 23%   37C    P8     8W / 250W |    115MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   32C    P8     8W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [4]:
# Imports
import os
import bz2
import collections
import math
import nltk
import operator
import random
import numpy as np
import tensorflow as tf
from six.moves import range
from six.moves.urllib.request import urlretrieve
import sklearn
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import scipy
from scipy.sparse import lil_matrix
from math import ceil
import matplotlib
from matplotlib import pylab
%matplotlib inline

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

sklearn          0.22.2
numpy            1.18.4
nltk             3.4.5
scipy            1.4.1
matplotlib.pylab 1.18.4
matplotlib       3.2.1
tensorflow       2.2.0
Data Science Academy


In [6]:
# Obs: Este script está compatível com as versões 1.x e 2.x do TensorFlow.
# Optamos por manter assim, pois alguns recursos avançados usados neste script ainda não foram implementados no TF 2.

# Para executar este script com TF 2, nenhum passo adicional precisa ser feito.
# Para executar com TF 1, remova o prefixo tf.compat.v1 ao longo do scriipt e substitua por tf, e comente as 3 linhas abaixo.
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.compat.v1.disable_eager_execution()

## Dataset
Este código faz o download de um [conjunto de dados] (http://www.evanjones.ca/software/wikipedia2text.html) que consiste em vários artigos da Wikipedia, totalizando aproximadamente 61 megabytes. Além disso, o código garante que o arquivo tenha o tamanho correto após o download.

In [7]:
url = 'http://www.evanjones.ca/software/'

def maybe_download(filename, expected_bytes):
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Encontrado e verificado %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Não foi possível verificar o arquivo ' + filename + '. Você consegue fazer o download via browser?')
  return filename

filename = 'dados/wikipedia2text-extracted.txt.bz2'

## Lendo dados com pré-processamento com NLTK
Lê os dados como estão em uma string, converte em minúscula e o converte em tokens usando a biblioteca nltk. Este código lê dados em porções de 1 MB (pois o processamento do texto completo de uma só vez pode ser lento) e retorna uma lista de palavras

In [8]:
def read_data(filename):

  with bz2.BZ2File(filename) as f:

    data = []
    file_size = os.stat(filename).st_size
    chunk_size = 1024 * 1024
    print('Lendo os dados...')
    for i in range(ceil(file_size//chunk_size)+1):
        bytes_to_read = min(chunk_size,file_size-(i*chunk_size))
        file_string = f.read(bytes_to_read).decode('utf-8')
        file_string = file_string.lower()
        file_string = nltk.word_tokenize(file_string)
        data.extend(file_string)
  return data

words = read_data(filename)
print('Tamanho do dataset %d' % len(words))
token_count = len(words)

print('Palavras de exemplo (start): ',words[:10])
print('Palavras de exemplo (end): ',words[-10:])

Lendo os dados...
Tamanho do dataset 3361041
Palavras de exemplo (start):  ['propaganda', 'is', 'a', 'concerted', 'set', 'of', 'messages', 'aimed', 'at', 'influencing']
Palavras de exemplo (end):  ['favorable', 'long-term', 'outcomes', 'for', 'around', 'half', 'of', 'those', 'diagnosed', 'with']


## Construindo os Dicionários
Para entender cada um desses elementos, vamos também assumir o texto "Eu gosto de ir à escola"

* `dictionary`: mapeia uma palavra para um ID (i.e. {Eu:0, gosto:1, de:2, ir:3, à:4, escola:5})
* `reverse_dictionary`: mapeia um ID para uma palavra (i.e. {0:Eu, 1:gosto, 2:de, 3:ir, 4:à, 5:escola}
* `count`: Lista de elementos (palavra, frequência) (i.e. [(Eu,1),(gosto,1),(de,2),(ir,1),(à,1),(escola,1)]
* `data` : Contém a string de texto que lemos, onde palavras são substituídas por IDs de palavras (i.e. [0, 1, 2, 3, 2, 4])

Também introduzimos um token especial adicional chamado `UNK` para indicar que palavras raras são muito raras para serem usadas.

In [9]:
# Nós restringimos o tamanho do nosso vocabulário para 50000
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]

  # Obtém apenas o vocabulary_size para palavras mais comuns como o vocabulário
  # Todas as outras palavras serão substituídas por token UNK
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()

  # Cria um ID para cada palavra, dando o comprimento atual do dicionário
  # e adicionando esse item ao dicionário
  for word, _ in count:
    dictionary[word] = len(dictionary)

  data = list()
  unk_count = 0

  # Percorre todo o texto que temos e produzir uma lista onde cada elemento corresponde ao ID
  # da palavra encontrada nesse índice
  for word in words:

    # Se a palavra estiver no dicionário, use a palavra ID, senão use o ID do token especial "UNK"
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)

  # Atualiza a variável count com o número de ocorrências UNK
  count[0][1] = unk_count

  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

  # Verifica se o dicionário é do tamanho do vocabulário
  assert len(dictionary) == vocabulary_size

  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Palavras mais comuns (+UNK)', count[:5])
print('Dados de amostra', data[:10])
del words

Palavras mais comuns (+UNK) [['UNK', 68859], ('the', 226892), (',', 184013), ('.', 120944), ('of', 116323)]
Dados de amostra [1721, 9, 8, 16476, 223, 4, 5166, 4457, 26, 11592]


## Gerando Lotes de Dados para o Data for GloVe
Gera um lote ou palavras de destino (`batch`) e um lote de palavras de contexto correspondentes (`labels`). Ele lê as palavras `2 * window_size + 1` por vez (chamado` span`) e cria os datapoints `2 * window_size` em um único intervalo. A função continua dessa maneira até que os datapoints `batch_size` sejam criados. Toda vez que chegamos ao final da sequência de palavras, retornamos ao começo.

In [10]:
data_index = 0

def generate_batch(batch_size, window_size):
  # data_index é atualizado por 1 toda vez que lemos um ponto de dados
  global data_index

  # Arrays para conter as palavras-alvo (lote), as palavras de contexto (etiquetas) e os pesos
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  weights = np.ndarray(shape=(batch_size), dtype=np.float32)

  # span define o tamanho total da janela, onde os dados que consideramos em uma instância
  # são exibidos da seguinte maneira: [skip_window target skip_window]
  span = 2 * window_size + 1

  # O buffer contém os dados contidos no intervalo
  buffer = collections.deque(maxlen=span)

  # Preenche o buffer e atualiza o data_index
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)

  # Este é o número de palavras de contexto que experimentamos para uma única palavra de destino
  num_samples = 2*window_size

  # Nós dividimos a leitura do lote em dois loops for
  # O loop interno preenche o lote e os rótulos com
  # num_samples pontos de dados usando dados contidos no intervalo
  # O loop externo é repetido por batch_size // num_samples times
  # para produzir um lote completo
  for i in range(batch_size // num_samples):
    k=0

    # Evita a própria palavra alvo como uma previsão
    # Preenchimento de matrizes de lotes e rótulos numpy
    for j in list(range(window_size))+list(range(window_size+1,2*window_size+1)):
      batch[i * num_samples + k] = buffer[window_size]
      labels[i * num_samples + k, 0] = buffer[j]
      weights[i * num_samples + k] = abs(1.0/(j - window_size))
      k += 1

    # Toda vez que lemos num_samples pontos de dados,
    # criamos o número máximo de pontos de dados possíveis
    # com um único intervalo, por isso precisamos mover o span por 1
    # para criar um novo intervalo
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels, weights

print('Dados:', [reverse_dictionary[di] for di in data[:8]])

for window_size in [2, 4]:
    data_index = 0
    batch, labels, weights = generate_batch(batch_size=8, window_size=window_size)
    print('\nCom window_size = %d:' %window_size)
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])
    print('    weights:', [w for w in weights])

Dados: ['propaganda', 'is', 'a', 'concerted', 'set', 'of', 'messages', 'aimed']

Com window_size = 2:
    batch: ['a', 'a', 'a', 'a', 'concerted', 'concerted', 'concerted', 'concerted']
    labels: ['propaganda', 'is', 'concerted', 'set', 'is', 'a', 'set', 'of']
    weights: [0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5]

Com window_size = 4:
    batch: ['set', 'set', 'set', 'set', 'set', 'set', 'set', 'set']
    labels: ['propaganda', 'is', 'a', 'concerted', 'of', 'messages', 'aimed', 'at']
    weights: [0.25, 0.33333334, 0.5, 1.0, 1.0, 0.5, 0.33333334, 0.25]


## Criando a Word Co-Occurance Matrix
Para o GloVe mostrar seu valor em relação ao método baseado em janela de contexto, ele emprega estatísticas globais do corpus no modelo. Isso é feito usando informações da matriz de co-ocorrência de palavras para otimizar os vetores de palavras. Basicamente, a entrada X (i, j) da matriz de co-ocorrência diz a frequência com que a palavra i aparece perto de j. Também usamos um mecanismo de ponderação para dar mais peso às palavras próximas umas das outras do que àquelas mais distantes.

In [11]:
# Estamos criando a matriz de co-ocorrência como uma matriz de coluna esparsa comprimida com scipy.
cooc_data_index = 0

# Iteramos pelo texto completo
dataset_size = len(data)

# Quantas palavras considerar à esquerda e à direita.
skip_window = 4

# A matriz esparsa que armazena a palavra co-ocorrências
cooc_mat = lil_matrix((vocabulary_size, vocabulary_size), dtype=np.float32)

print(cooc_mat.shape)
def generate_cooc(batch_size,skip_window):
    '''
    Gerar matriz de co-ocorrência processando lotes de dados
    '''
    data_index = 0
    print('Executando %d iterações para computar a co-occurance matrix'%(dataset_size//batch_size))
    for i in range(dataset_size//batch_size):
        # Imprimindo progresso
        if i>0 and i%100000==0:
            print('\tConcluídas %d iterações'%i)

        # Gerando um único lote de dados
        batch, labels, weights = generate_batch(batch_size, skip_window)
        labels = labels.reshape(-1)

        # Incrementando as entradas de matriz esparsas de acordo
        for inp,lbl,w in zip(batch,labels,weights):
            cooc_mat[inp,lbl] += (1.0*w)

# Gera a matriz
generate_cooc(8,skip_window)
print('Chunks de exemplo da co-occurance matrix')


# Basicamente calcula o co-occurance mais alto de várias palavras escolhidas
for i in range(10):
    idx_target = i

    # Obter a linha i da matriz esparsa e torná-lo denso
    ith_row = cooc_mat.getrow(idx_target)
    ith_row_dense = ith_row.toarray('C').reshape(-1)

    # Seleciona palavras-alvo apenas com palavras razoáveis ao redor.
    while np.sum(ith_row_dense)<10 or np.sum(ith_row_dense)>50000:
        idx_target = np.random.randint(0,vocabulary_size)

        # Obter a linha i da matriz esparsa e torná-la densa
        ith_row = cooc_mat.getrow(idx_target)
        ith_row_dense = ith_row.toarray('C').reshape(-1)

    print('\nPalavra alvo: "%s"'%reverse_dictionary[idx_target])

    # Índices com maior contagem de ith_row_dense
    sort_indices = np.argsort(ith_row_dense).reshape(-1)

    # Inverter a matriz (para obter valores máximos para o início)
    sort_indices = np.flip(sort_indices,axis=0)

    # Imprimindo várias palavras de contexto para garantir que cooc_mat esteja correto
    print('Context word:',end='')
    for j in range(10):
        idx_context = sort_indices[j]
        print('"%s"(id:%d,count:%.2f), '%(reverse_dictionary[idx_context],idx_context,ith_row_dense[idx_context]),end='')
    print()

(50000, 50000)
Executando 420130 iterações para computar a co-occurance matrix
	Concluídas 100000 iterações
	Concluídas 200000 iterações
	Concluídas 300000 iterações
	Concluídas 400000 iterações
Chunks de exemplo da co-occurance matrix

Palavra alvo: "UNK"
Context word:","(id:2,count:3243.73), "UNK"(id:0,count:2020.35), "the"(id:1,count:1982.02), "and"(id:5,count:1413.50), "."(id:3,count:1284.67), "of"(id:4,count:994.16), "("(id:13,count:960.42), "in"(id:6,count:771.67), ")"(id:12,count:762.33), "a"(id:8,count:549.17), 

Palavra alvo: "1st"
Context word:"the"(id:1,count:11.00), ","(id:2,count:6.75), "."(id:3,count:5.08), "century"(id:92,count:4.00), "edn"(id:23000,count:3.00), "in"(id:6,count:2.58), "of"(id:4,count:2.08), "("(id:13,count:1.83), "duke"(id:1217,count:1.50), "bc"(id:568,count:1.50), 

Palavra alvo: "algeria"
Context word:","(id:2,count:7.33), "."(id:3,count:5.17), "in"(id:6,count:3.83), "of"(id:4,count:2.83), "for"(id:15,count:2.33), "UNK"(id:0,count:2.33), "has"(id:35,co

## Algoritmo GloVe 

### Definindo Hiperparâmetros

Aqui nós definimos vários hiperparâmetros incluindo:

- `batch_size` (quantidade de amostras em um único lote) 
- `embedding_size` (tamanho dos embedding vectors) 
- `window_size` (tamanho da janela de contexto).

In [12]:
# Pontos de dados em um único lote
batch_size = 128

# Dimensão do embedding vector.
embedding_size = 128

# Quantas palavras considerar à esquerda e à direita.
window_size = 4

# Nós escolhemos um conjunto de validação aleatório para testar os vizinhos mais próximos
# Conjunto aleatório de palavras para avaliar a similaridade.
valid_size = 16

# Nós experimentamos datapoints válidos aleatoriamente a partir de uma janela grande sem sermos sempre determinísticos
valid_window = 50

# Ao selecionar exemplos válidos, selecionamos algumas das palavras mais frequentes,
# bem como algumas palavras moderadamente raras
valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.append(valid_examples,random.sample(range(1000, 1000+valid_window), valid_size),axis=0)

# Número de exemplos negativos para amostra.
num_sampled = 32

# Usado para a estabilidade do log na função de perda
epsilon = 1

### Definindo Entradas e Saídas

Aqui nós definimos espaços reservados (placeholders) para alimentação dos dados de entrada e saída para o treinamento (cada um de tamanho `batch_size`) e um tensor constante para conter exemplos de validação.

In [13]:
tf.compat.v1.reset_default_graph()

# Dados de entrada de treinamento (IDs de palavras de destino).
train_dataset = tf.compat.v1.placeholder(tf.int32, shape=[batch_size])

# Dados de labels de entrada de treinamento (IDs de palavras de contexto)
train_labels = tf.compat.v1.placeholder(tf.int32, shape=[batch_size])

# Dados de entrada de validação, não precisamos de um espaço reservado,
# pois já definimos os IDs das palavras selecionadas como dados de validação
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

### Definindo Parâmetros do Modelo e Outras Variáveis
Nós agora definimos diversas variáveis do TensorFlow, como uma camada embeddings (`embeddings`) e parâmetros de rede neural (`softmax_weights` e `softmax_biases`)

In [14]:
# Variáveis
in_embeddings = tf.Variable(
    tf.random.uniform([vocabulary_size, embedding_size], -1.0, 1.0),name='embeddings')

in_bias_embeddings = tf.Variable(tf.random.uniform([vocabulary_size],0.0,0.01,dtype=tf.float32),name='embeddings_bias')

out_embeddings = tf.Variable(
    tf.random.uniform([vocabulary_size, embedding_size], -1.0, 1.0),name='embeddings')

out_bias_embeddings = tf.Variable(tf.random.uniform([vocabulary_size],0.0,0.01,dtype=tf.float32),name='embeddings_bias')

### Definindo as Computações Modelo

Primeiro, definimos uma função de pesquisa para buscar os vetores embedding correspondentes para um conjunto de entradas fornecidas. Em seguida, definimos um espaço reservado que aceita os pesos de um determinado lote de pontos de dados (`weights_x`) e pesos de matriz de co-ocorrência (`x_ij`). `weights_x` mede a importância de um ponto de dados em relação ao quanto essas duas palavras co-ocorrem e `x_ij` indica o valor da matriz de co-ocorrência para a linha e coluna indicadas pelas palavras em um ponto de dados. Com estes definidos, podemos definir a perda como mostrado abaixo. 

In [15]:
# Procura por embeddings para entradas e saídas
# Temos dois espaços vetoriais de inserção separados para entradas e saídas
embed_in = tf.nn.embedding_lookup(params=in_embeddings, ids=train_dataset)
embed_out = tf.nn.embedding_lookup(params=out_embeddings, ids=train_labels)
embed_bias_in = tf.nn.embedding_lookup(params=in_bias_embeddings,ids=train_dataset)
embed_bias_out = tf.nn.embedding_lookup(params=out_bias_embeddings,ids=train_labels)

# Pesos usados na função de custo
weights_x = tf.compat.v1.placeholder(tf.float32,shape=[batch_size],name='weights_x')

# Valor de co-ocorrência para cada posição
x_ij = tf.compat.v1.placeholder(tf.float32,shape=[batch_size],name='x_ij')

# Calcula a perda definida no paper do algoritmo.
# Observe que eu não estou seguindo a equação exata dada (que está computando um par de palavras de cada vez)
# Estou calculando a perda de um lote de uma só vez, mas os cálculos são idênticos.
# Eu também fiz uma suposição sobre o viés, que é um tipo menor de embedding
loss = tf.reduce_mean(
    input_tensor=weights_x * (tf.reduce_sum(input_tensor=embed_in*embed_out,axis=1) + embed_bias_in + embed_bias_out - tf.math.log(epsilon+x_ij))**2)

### Calculando Similaridade de palavras
Calculamos a similaridade entre duas palavras dadas em termos da distância do cosseno. Para fazer isso de maneira eficiente, usamos operações de matriz para fazer isso, conforme mostrado abaixo.

In [16]:
# Calcula a similaridade entre os exemplos de minibatch e todos os embeddings.
# Nós usamos a distância do cosseno:
embeddings = (in_embeddings + out_embeddings)/2.0
norm = tf.sqrt(tf.reduce_sum(input_tensor=tf.square(embeddings), axis=1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(
params=normalized_embeddings, ids=valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(a=normalized_embeddings))

### Otimizador de Parâmetros do Modelo

Em seguida, definimos uma taxa de aprendizado constante e um otimizador que usa o método Adagrad. Sinta-se à vontade para experimentar outros otimizadores listados [aqui] (https://www.tensorflow.org/api_guides/python/train).

In [17]:
# Otimizador
optimizer = tf.compat.v1.train.AdagradOptimizer(1.0).minimize(loss)

## Treinando o Algoritmo GloVe 

Aqui nós executamos o algoritmo GloVe que definimos acima. Especificamente, primeiro inicializamos as variáveis e depois treinamos o algoritmo para várias etapas (`num_steps`). E a cada poucos passos avaliamos o algoritmo em um conjunto de validação fixo e imprimimos as palavras que parecem estar mais próximas de um determinado conjunto de palavras.

In [18]:
num_steps = 100001
glove_loss = []

average_loss = 0
with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(allow_soft_placement=True)) as session:

    tf.compat.v1.global_variables_initializer().run()
    print('Variáveis Inicializadas')

    for step in range(num_steps):

        # Gera um único lote de dados (data,labels,co-occurance weights)
        batch_data, batch_labels, batch_weights = generate_batch(batch_size, skip_window)

        # Calculando os pesos exigidos pela função de perda
        # Ponderação usada na função de perda
        batch_weights = []

        # Frequência ponderada de encontrar perto de j
        batch_xij = []

        # Calcular os pesos para cada ponto de dados no lote
        for inp,lbl in zip(batch_data,batch_labels.reshape(-1)):
            point_weight = (cooc_mat[inp,lbl]/100.0)**0.75 if cooc_mat[inp,lbl]<100.0 else 1.0
            batch_weights.append(point_weight)
            batch_xij.append(cooc_mat[inp,lbl])
        batch_weights = np.clip(batch_weights,-100,1)
        batch_xij = np.asarray(batch_xij)

        # Preencha o feed_dict e execute o otimizador (minimize a perda) e calcule a perda.
        # Especificamente nós fornecemos train_dataset / train_labels: treinamento de entradas e rótulos de treinamento
        # weights_x: mede a importância de um ponto de dados em relação ao quanto essas duas palavras co-ocorrem
        # x_ij: valor da matriz de co-ocorrência para a linha e coluna indicadas pelas palavras em um ponto de dados
        feed_dict = {train_dataset : batch_data.reshape(-1), train_labels : batch_labels.reshape(-1),
                    weights_x:batch_weights,x_ij:batch_xij}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)

        # Atualiza a variável de perda média
        average_loss += l
        if step % 2000 == 0:
          if step > 0:
            average_loss = average_loss / 2000

          # A perda média é uma estimativa da perda nos últimos 2000 lotes.
          print('Perda média no passo %d: %f' % (step, average_loss))
          glove_loss.append(average_loss)
          average_loss = 0

        # Aqui calculamos as palavras top_k mais próximas para uma determinada palavra de validação
        # em termos da distância do coseno
        # Fazemos isso para todas as palavras no conjunto de validação
        if step % 10000 == 0:
          sim = similarity.eval()
          for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8 # number of nearest neighbors
            nearest = (-sim[i, :]).argsort()[1:top_k+1]
            log = 'Palavra Mais Próxima de %s:' % valid_word
            for k in range(top_k):
              close_word = reverse_dictionary[nearest[k]]
              log = '%s %s,' % (log, close_word)
            print(log)

    final_embeddings = normalized_embeddings.eval()


Variáveis Inicializadas
Perda média no passo 0: 16.888613
Palavra Mais Próxima de that: mountain, vaporized, 1952., pseudomonas, 1/3rd, vice-versa, left-leaning, porridge,
Palavra Mais Próxima de not: chaktomuk, mottes, hotspot, metrics, chōng, outfitted, embarrassed, bolivian,
Palavra Mais Próxima de its: prophesied, grudge, serie, pinpointed, onstar, 1994., patched, jealousy,
Palavra Mais Próxima de .: are, also, the, some, and, in, splendor, contiguous,
Palavra Mais Próxima de be: ugarte, tibetans, salle, burner, rhythm, emetic, 147, convened,
Palavra Mais Próxima de but: sieur, yamashiro, deportations, mihail, accomplish, thulborn, fletcher, jamming,
Palavra Mais Próxima de were: turboprop, voc, balboa, insofar, volatility, softer, drawer, green-yellow,
Palavra Mais Próxima de (: localised, plunkett, overgrazing, fares, mikoyan, potatoes, chop, jonestown,
Palavra Mais Próxima de ,: and, of, some, memories, courtesy, volatiles, barrel, sky,
Palavra Mais Próxima de was: portuguese-sp

Perda média no passo 62000: 0.023637
Perda média no passo 64000: 0.023344
Perda média no passo 66000: 0.023063
Perda média no passo 68000: 0.022504
Perda média no passo 70000: 0.022838
Palavra Mais Próxima de that: is, was, to, the, ., by, it, ,,
Palavra Mais Próxima de not: it, that, is, was, were, but, be, which,
Palavra Mais Próxima de its: for, and, the, ,, in, to, on, from,
Palavra Mais Próxima de .: the, in, of, and, ,, 's, a, for,
Palavra Mais Próxima de be: to, have, not, can, a, from, for, is,
Palavra Mais Próxima de but: which, not, with, ,, and, was, it, a,
Palavra Mais Próxima de were: are, was, by, and, ,, in, they, to,
Palavra Mais Próxima de (: ), UNK, '', or, ,, and, ``, .,
Palavra Mais Próxima de ,: and, in, the, UNK, ., a, with, of,
Palavra Mais Próxima de was: is, by, a, in, ., it, ,, that,
Palavra Mais Próxima de a: ,, is, and, ., with, the, for, in,
Palavra Mais Próxima de ): (, UNK, '', ,, and, or, in, .,
Palavra Mais Próxima de 's: the, ., of, in, on, and, ,, fro

## Fim